In [1]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2000000)

Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (2000000 rows)...
Finished inserting 2000000 rows into 'products' in 35.39s.


The purpose of this is to display and understand why the order of the where filters matter when you index spefically with a range involved.
So in this case we will filter on the price and category id. This could be arbitary just as an example.

Initialy i would have the assumtion if we didn't have any index then the fast query would be the one where we only do where category_id = 5. Because i would assume that this would be easier to find because it does not have to do any computation to check if the price is between a range

In [2]:
query = """
    SELECT name, price FROM products WHERE price BETWEEN 100 AND 200;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

[QUERY METRICS] 201548 rows fetched, 355.49 ms


In [3]:
query = """
    SELECT name, price FROM products WHERE category_id = 5;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

[QUERY METRICS] 58961 rows fetched, 242.54 ms


In [4]:
query = """SELECT name, price FROM products WHERE price BETWEEN 100 AND 200 and category_id = 5;"""
execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)


[QUERY METRICS] 6015 rows fetched, 238.13 ms


What i found out was the the latter where you include both where price between and cateogory_id = is the fastest. And this did not initially make sense to me because i thought that the between would be more costly. 

What i know think is the case is that the acutal search is very small and what ends up making this faster is that the amount of data to fetch/blocks to read is the actual bottle neck and there by making it faster

_______

Lets run an explain on all of them

In [5]:
query = """
    EXPLAIN SELECT name, price FROM products WHERE price BETWEEN 100 AND 200;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,1982158,11.11,Using where


In [6]:
query = """
    EXPLAIN SELECT name, price FROM products WHERE category_id = 5;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=True)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,category_id,category_id,5,const,103074,100.0,None


[QUERY METRICS] 1 rows fetched, 0.87 ms


In [7]:
query = """EXPLAIN SELECT name, price FROM products WHERE price BETWEEN 100 AND 200 and category_id = 5;"""
execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,category_id,category_id,5,const,103074,11.11,Using where


From this explain i wondered why the type was ref for the other two. and that had to do with the foregin key. Because eventhough i have not created a index myself for this tabe because the table is has a primary key i think automatically MySQl will reuse this index or something like that 

In [8]:
show_table_indexes("products", get_database_name())

,Index Name,Columns,Type,Non Unique
0,availability_id,availability_id,BTREE,True
1,brand_id,brand_id,BTREE,True
2,category_id,category_id,BTREE,True
3,color_id,color_id,BTREE,True
4,PRIMARY,id,BTREE,False


Would this all look different if no foregin key is involved?

In [9]:
query = """
    SELECT name FROM products WHERE price BETWEEN 100 AND 200;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 201548 rows fetched, 354.61 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,1982158,11.11,Using where


In [10]:
query = """
    SELECT name FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 111336 rows fetched, 368.20 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,1982158,10.0,Using where


In [11]:
query = """
    SELECT name, price FROM products WHERE price BETWEEN 100 AND 200 AND size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 11222 rows fetched, 386.37 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,1982158,1.11,Using where


This time they are all full scan and my initial assumption seems to acutally hold true.

Lets when moving forward stick to using the Size instead of category id

This time lets create a index for both of the columns and run the same test

In [12]:
query = "CREATE INDEX idx_price ON products (price)"

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

query = "CREATE INDEX idx_size ON products (size)"
execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

[QUERY METRICS] 0 rows affected, 895.29 ms
[QUERY METRICS] 0 rows affected, 1435.69 ms


In [13]:
execute_query("SELECT name, price, size FROM products where price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=True, show_metrics=False)


execute_query("SELECT name, price, size FROM products where size = 'Large'", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where size = 'Large'", database=get_database_name(), print_as_df=True, show_metrics=False)


execute_query("SELECT name, price, size FROM products where price BETWEEN 100 AND 200 and size = 'Large'", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where price BETWEEN 100 AND 200 and size = 'Large'", database=get_database_name(), print_as_df=True, show_metrics=False)


execute_query("SELECT name, price, size FROM products where size = 'Large' and price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where size = 'Large' and price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 201548 rows fetched, 432.33 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,idx_price,None,None,None,1982158,20.13,Using where


[QUERY METRICS] 111336 rows fetched, 279.25 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_size,idx_size,403,const,223618,100.0,None


[QUERY METRICS] 11222 rows fetched, 249.94 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,"idx_price,idx_size",idx_size,403,const,223618,20.13,Using where


[QUERY METRICS] 11222 rows fetched, 247.67 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,"idx_price,idx_size",idx_size,403,const,223618,20.13,Using where


I read this article: https://use-the-index-luke.com/sql/where-clause/searching-for-ranges/greater-less-between-tuning-sql-access-filter-predicates

and from this i expected there to be a difference between the last two quries but there isn't because it seems that mysql pick the right key so that is a bit curious i think. so just to really test if there is a difference lets try to drop the index on the size

In [14]:
execute_query("DROP index idx_size ON products", database=get_database_name(), print_as_df=False, show_metrics=True)

[QUERY METRICS] 0 rows affected, 4.68 ms


In [18]:
clear_mysql_cache()

query = "SELECT name, price, size FROM products where price BETWEEN 100 AND 110"

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

MySQL cache cleared
[QUERY METRICS] 21868 rows fetched, 450.41 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,range,idx_price,idx_price,5,None,40534,100.0,Using index condition; Using MRR


In [20]:
clear_mysql_cache()

query = "SELECT name, price, size FROM products where price BETWEEN 100 AND 200"

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

MySQL cache cleared
[QUERY METRICS] 201548 rows fetched, 447.59 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,idx_price,None,None,None,1982158,20.13,Using where


This is something fun and Quirky it seems that the MySql execution optimizer will weight that because it guesses that a high amount of the rows will match the query then it will use a full table scan. But if it things that not that many rows will match the filter then it will use the key. 